In this notebook, I want to do some analysis for question 1 and question 3 for the Baby Names assignment.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

#Read in Data
q1data = pd.read_csv(r"C:\Users\cathe\OneDrive\Documents\Portfolio Data\DCbabynames\filtered_names.csv")
q3data = pd.read_csv(r"C:\Users\cathe\OneDrive\Documents\Portfolio Data\DCbabynames\summed_occurrences_by_decade2.csv")

In [2]:
q1data.head()

,state,gender,year,name,occurrences
0,AK,F,2012,Charlie,9
1,AK,F,2014,Charlie,6
2,AK,F,2015,Charlie,6
3,AK,F,2016,Charlie,9
4,AK,F,2017,Charlie,7


I would like to create gender dummies to analyze the categorical data.  Then I want to split up the table ito one for just Charlie and one for just Oakley.  I want to look at the data in two ways to see if the names are more associate with one gender or the other.  First, I want to look at the correlation between occurences and gender.  Then, I want to look at crosstabs.

In [3]:
gender_dm = pd.get_dummies(q1data.gender, prefix='gender')
q1data = pd.concat([q1data,gender_dm], axis=1)

In [4]:
q1data.head()

,state,gender,year,name,occurrences,gender_F,gender_M
0,AK,F,2012,Charlie,9,1,0
1,AK,F,2014,Charlie,6,1,0
2,AK,F,2015,Charlie,6,1,0
3,AK,F,2016,Charlie,9,1,0
4,AK,F,2017,Charlie,7,1,0


In [5]:
import math
import scipy.stats as stats
from scipy.stats import chi2_contingency
from scipy.stats import norm    

In [6]:
Charlie = 'Charlie'

In [7]:
charlie = q1data[q1data['name'] == Charlie].copy()

In [8]:
charlie.head()

,state,gender,year,name,occurrences,gender_F,gender_M
0,AK,F,2012,Charlie,9,1,0
1,AK,F,2014,Charlie,6,1,0
2,AK,F,2015,Charlie,6,1,0
3,AK,F,2016,Charlie,9,1,0
4,AK,F,2017,Charlie,7,1,0


In [9]:
charlie.corr()

C:\Users\cathe\AppData\Local\Temp\ipykernel_41712\1782768021.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  charlie.corr()


,year,occurrences,gender_F,gender_M
year,1.000000,-0.223944,0.218887,-0.218887
occurrences,-0.223944,1.000000,-0.165667,0.165667
gender_F,0.218887,-0.165667,1.000000,-1.000000
gender_M,-0.218887,0.165667,-1.000000,1.000000


In [10]:
Oakley = 'Oakley'
oakley = q1data[q1data['name'] == Oakley].copy()
oakley.head()

,state,gender,year,name,occurrences,gender_F,gender_M
10,AK,F,2022,Oakley,5,1,0
110,AL,F,2012,Oakley,6,1,0
113,AL,F,2014,Oakley,7,1,0
115,AL,F,2015,Oakley,7,1,0
117,AL,F,2016,Oakley,9,1,0


In [11]:
oakley.corr()

C:\Users\cathe\AppData\Local\Temp\ipykernel_41712\1033452852.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  oakley.corr()


,year,occurrences,gender_F,gender_M
year,1.000000,0.224915,0.222692,-0.222692
occurrences,0.224915,1.000000,0.264127,-0.264127
gender_F,0.222692,0.264127,1.000000,-1.000000
gender_M,-0.222692,-0.264127,-1.000000,1.000000


These correlation tables tell us there's a slightly higher correlation of Charlie being a male name and Oakley being a female name.  Oakley has a positive correlation of .26 with being Female, while Charlie has a negative correlation of -.16 of being Female.  So Oakley is more female associated, while Charlie is more male associated.  We can also see a few other trends in the data.  For Oakley, there is a positive correlation between year and occurrences, meaning that there are more people named Oakley as time goes on, but there's a negative correlation between year and occurrences for Charlie, so there are fewer Charlies.  We also see a positive correlation between year and Females in the Charlie table, meaning that while for the entire dataset there are more male Charlie's than female, the number of female Charlies is going up over time, and the same is true for Oakley's.

However, this doesn't tell us the level of significance when it comes to the association of these names and gender.  One way to do that would be a chi squared test, we would need to unpivot this data into individual rows because chi squared tests are for categorical data, not numeric.  We already have an answer, but we can still do it and look at the crosstabs.

In [14]:
q1data_unpvt = q1data.loc[q1data.index.repeat(q1data.occurrences)]

In [15]:
from scipy.stats import chi2_contingency    

In [16]:
name_dm = pd.get_dummies(q1data_unpvt.name, prefix='name')
q1data_unpvt = pd.concat([q1data_unpvt,name_dm], axis=1)

Here, we have the labelled crosstabs with percentages by row.  83% of Charlies in the dataset are male, while only 16% are female.  Oakley is far more gender neutral with 57% female and 42% male.  These findings are consistent with what we saw from the correlation matrix, but I think they add a little more color and magnitude to the data for this specific question.  What I like about the correlation matrix is that it showed me the interactions between other elements of the dataset, so I would still do both in the future. 

In [17]:
contingency1= pd.crosstab(q1data_unpvt['name'],q1data_unpvt['gender'], normalize='index')
print(contingency1)

gender          F         M
name                       
Charlie  0.164786  0.835214
Oakley   0.574588  0.425412


To do the Chi Squared test for significance, I need to code the names as dummies and run the crosstabs again.  

In [20]:
contingency2= pd.crosstab(q1data_unpvt['name_Oakley'],q1data_unpvt['gender_F'])
print(contingency2)

gender_F          0      1
name_Oakley               
0            151431  29877
1              6482   8755


In [21]:
c, p, dof, expected = chi2_contingency(contingency2)
p

0.0

The p value is 0, so it looks like these directional findings are statistically significant.  

Now I want to see if I can find something fun and interesting in the data.  I put together a table with the names of main characters from the top 3 TV shows in each decade from 1950 to 2010's, then I made a binary variable that is a 1 for each record where a given name was in the top 10 for it's decade.  For example, I love Lucy was the top show in the 1950's, so Lucy in 1955 would have a 1, but Lucy in 1975 would have a 0. 

In [25]:
q3data.head()

,name,decade,occurrences,is_in_tv_names
0,Abby,1990,13692,1
1,Adam,1960,22778,1
2,Adam,1980,195513,1
3,Albert,1970,22003,1
4,Alexis,1980,30832,1


In [26]:
descriptive_stats = q3data.describe()
print(descriptive_stats)

              decade    occurrences  is_in_tv_names
count  114907.000000  114907.000000   114907.000000
mean     1978.139626    2807.304664        0.000870
std        32.390179   19551.614146        0.029488
min      1910.000000       5.000000        0.000000
25%      1960.000000      11.000000        0.000000
50%      1990.000000      50.000000        0.000000
75%      2000.000000     360.500000        0.000000
max      2020.000000  845868.000000        1.000000


In [27]:
import statsmodels.api as sm

# Create a sample DataFrame (replace this with your actual data)
# Add a constant term to the independent variable for the intercept
X = sm.add_constant(q3data['is_in_tv_names'])

# Fit the linear regression model
model = sm.OLS(q3data['occurrences'], X)
result = model.fit()

# Display the regression results
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:            occurrences   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     446.5
Date:                Sat, 18 Nov 2023   Prob (F-statistic):           6.37e-99
Time:                        20:06:21   Log-Likelihood:            -1.2982e+06
No. Observations:              114907   AIC:                         2.596e+06
Df Residuals:                  114905   BIC:                         2.596e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           2771.4038     57.591     48.

Interesting results!  I'd love to explore this more in PowerBI!  According to this, there are over 40,000 more occurences of a name if it's the name of a main character in a top tv show of a given decade! I wouldn't put much weight into that number because there are so many other potential factors here. 